<a href="https://colab.research.google.com/github/DT6A/Hands-On_Machine_Learning/blob/main/ML11_Distribution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

#Multiple devices on a single machine

##Placing operations

In [ ]:
! nvidia-smi

Mon Jul  1 19:30:43 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P8    17W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [ ]:
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.4
session = tf.Session(config=config)

In [ ]:
session.close()

###Simple placement

In [ ]:
with tf.device('/cpu:0'):
  a = tf.Variable(3.0)
  b = tf.constant(4.0)
c = a * b

In [ ]:
sess = tf.Session()

a.initializer.run(session=sess)

In [ ]:
sess.run(c)

12.0

In [ ]:
sess.close()

### Dynamic placement function

In [ ]:
def variables_on_cpu(op):
  if op.type == 'Variable':
    return '/cpu:0'
  else:
    return '/gpu:0'
  
with tf.device(variables_on_cpu):
  a = tf.Variable(3.0)
  b = tf.constant(4.0)
  c = a * b

### Operations and kernels

Not all operations have ***kernel*** for GPU!

In [ ]:
with tf.device('/gpu:0'):
  i = tf.Variable(3)
with tf.Session() as sess:
  sess.run(i.initializer)

InvalidArgumentError: ignored

### Soft placement

In [ ]:
with tf.device('/gpu:0'):
  i = tf.Variable(3)
  
config = tf.ConfigProto()
config.allow_soft_placement = True

with tf.Session(config=config) as sess:
  sess.run(i.initializer)

##Control Dependencies

In [ ]:
tf.reset_default_graph()

a = tf.constant(1.0)
b = a + 2.0

with tf.control_dependencies([a, b]):
  x = tf.constant(3.0)
  y = tf.constant(4.0)
  
z = x + y

In [ ]:
with tf.Session() as sess:
  z.eval(session=sess)

#Multiple Devices Across Multiples Servers

In [ ]:
cluster_spec = tf.train.ClusterSpec({
    'ps':[
        'machine-a.example.com:2221', #/job:ps/task:0
    ],
    'worker':[
        'machine-a.example.com:2222', #/job:worker/task:0
        'machine-b.example.com:2222', #/job:worker/task:1
    ]
})

In [ ]:
server = tf.train.Server(cluster_spec, job_name='worker', task_index=0)

In [ ]:
server.join() # blocks until the server stops

In [ ]:
a = tf.constant(1.0)
b = a + 2
c = a * 3

with tf.Session('grpc://machine-b.example.com:2222') as sess:
  print(c.eval())

In [ ]:
with tf.device('/job:ps/task:0/cpu:0'):
  a = tf.constant(1.0)
with tf.device('/job:worker/task:0/gpu:1'):
  b = a + 2
c = a + b

In [ ]:
# Sharing variables

with tf.device(tf.train.replica_device_setter(ps_tasks=2)):
  v1 = tf.Variable(1.0) # /job:ps/task:0
  v2 = tf.Variable(2.0) # /job:ps/task:1
  v3 = tf.Variable(3.0) # /job:ps/task:0
  v4 = tf.Variable(4.0) # /job:ps/task:1
  v5 = tf.Variable(5.0) # /job:ps/task:0

In [ ]:
with tf.device(tf.train.replica_device_setter(ps_tasks=2)):
  v1 = tf.Variable(1.0) # /job:ps/task:0 (default /cpu:0)
  v2 = tf.Variable(2.0) # /job:ps/task:1 (default /cpu:0)
  v3 = tf.Variable(3.0) # /job:ps/task:0 (default /cpu:0)
  s = v1 + v2 # /job:worker (default task:0/gpu:0)
  with tf.device('/gpu:1'):
    p1 = 2 * s # /job:worker/gpu1 (default task:0)
    with tf.device('/task:1'):
      p2 = 3 * s # /job:worker/task:1/gpu:1

In [ ]:
# simple_client.py
import tensorflow as tf
import sys

x = tf.Variable(0.0, name='x')
increment_x = tf.assign(x, x + 1)

with tf.Session(sys.argv[1]) as sess:
  if sys.argv[2:] == ['init']:
    sess.run(x.initializer)
  sess.run(increment_x)
  print(x.eval())

In [ ]:
! python3 simple_client.py grpc://machine-a.example.com:2222 init
#outputs 1

In [ ]:
! python3 simple_client.py grpc://machine-b.example.com:2222
#outputs 2

In [ ]:
with tf.container('my_problem_1'):
  # Construction phase of problem 1

In [ ]:
tf.Session().reset('grpc://machine-a.example.com:2222',['my_problem_1'])

In [ ]:
q = tf.FIFOQueue(capacity=10, dtypes=[tf.float32], shapes=[[2]], name='q', shared_name='shared_q')

In [ ]:
# training_data_loader.py
import tensorflow as tf

q = tf.FIFOQueue(capacity=10, dtypes=[tf.float32], shapes=[[2]], name='q', shared_name='shared_q')
training_instances = tf.placeholder(tf.float32, shape=[2])
enqueue = q.enqueue([training_instances])

with tf.Session('grpc://machine-a.example.com:2222') as sess:
  sess.run(enqueue, feed_dict={training_instances:[1., 2.]})
  sess.run(enqueue, feed_dict={training_instances:[3., 4.]})
  sess.run(enqueue, feed_dict={training_instances:[5., 6.]})
  
training_instances = tf.placeholder(tf.float32, shape=(None, 2))
enqueue_many = q.enqueue_many([training_instances])

with tf.Session('grpc://machine-a.example.com:2222') as sess:
  sess.run(enqueue_many, feed_dict={training_instances: [[1., 2.], [3., 4.], [5., 6.]]})

In [ ]:
# trainer.py
import tensorflow as tf

q = tf.FIFOQueue(capacity=10, dtypes=[tf.float32], shapes=[[2]], name='q', shared_name='shared_q')
dequeue = q.dequeue()

with tf.Session('grpc://machine-a.example.com:2222') as sess:
  print(sess.run(dequeue)) # [1., 2.]
  print(sess.run(dequeue)) # [3., 4.]
  print(sess.run(dequeue)) # [5., 6.]
  
batch_size = 2
dequeue_mini_batch = q.dequeue_many(batch_size)

with tf.Session('grpc://machine-a.example.com:2222') as sess:
  print(sess.run(dequeue_mini_batch)) # [[1., 2.], [3., 4.]]
  print(sess.run(dequeue_mini_batch)) # blocked


In [ ]:
q = tf.FIFOQueue(capacity=10, dtypes=[tf.int32, tf.float32], shapes=[[], [3, 2]], name='q', shared_name='shared_q')

a = tf.placeholder(tf.int32, shape=())
b = tf.placeholder(tf.float32, shape=(3, 2))

enqueue = q.enqueue((a, b))

with tf.Session() as sess:
  sess.run(enqueue, feed_dict={a: 10, b:[[1., 2.], [3., 4.], [5., 6.]]})
  
dequeue_a, dequeue_b = q.dequeue()

with tf.Session() as sess:
  a_val, b_val = sess.run([dequeue_a, dequeue_b])
  print(a_val) # 10
  print(b_val) # [[1., 2.], [3., 4.], [5., 6.]]
  

batch_size = 2
dequeue_as, dequeue_bs = q.dequeue_many(batch_size)

with tf.Session() as sess:
  a, b = sess.run([dequeue_as, dequeue_bs])

In [ ]:
close_q = q.close()

with tf.Session() as sess:
  sess.run(close_q)

## Loading Data

In [ ]:
training_set_init = tf.placeholder(tf.float32, shape=(None, n_features))
training_set = tf.Variable(training_set_init, trainable=False, collections=[], name='training_set')

with tf.Session() as sess:
  data = [...] # load data
  sess.run(training_set.initializer, feed_dict={training_set_init: data})

In [ ]:
reader = tf.TextLineReader(skip_header_lines=1)

filename_queue = tf.FIFOQueue(capacity=10, dtypes=[tf.string], shapes=[()])
filename = tf.placeholder(tf.string)
enqueue_filename = filename_queue.enqueue([filename])
close_filename_queue = filename_queue.close()

W0702 17:54:39.677659 140615820392320 deprecation.py:323] From <ipython-input-2-cb1935fc0121>:1: TextLineReader.__init__ (from tensorflow.python.ops.io_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.TextLineDataset`.


In [ ]:
key, value = reader.read(filename_queue)

In [ ]:
x1, x2, target = tf.decode_csv(value, record_defaults=[[-1.], [-1.], [-1]])
features = tf.stack([x1, x2])

In [ ]:
instance_queue = tf.RandomShuffleQueue(capacity=10, min_after_dequeue=2, dtypes=[tf.float32, tf.int32], shapes=[[2], []],
                                       name='instance_q', shared_name='shared_instance_q')
enqueue_instance = instance_queue.enqueue([features, target])
close_instance_queue = instance_queue.close()
                                   

In [ ]:
with tf.Session() as sess:
  sess.run(enqueue_filename, feed_dict={filename: 'my_test.csv'})
  sess.run(close_filename_queue)
  try:
    while True:
      sess.run(enqueue_instance)
  except tf.errors.OutOfRangeError as ex:
    pass
  sess.run(close_instance_queue)

In [ ]:
instance_queue = tf.RandomShuffleQueue(capacity=10, min_after_dequeue=2, dtypes=[tf.float32, tf.int32], shapes=[[2], []],
                                       name='instance_q', shared_name='shared_instance_q')
mini_batch_instances, mini_batch_targets = instance_queue.dequeue_up_to(2)

[...] # building graph

training_op = [...]

with tf.Session([...]) as sess:
  try:
    for step in range(max_steps):
      sess.run(training_op)
  except tf.errors.OutOfRangeError as ex:
    pass

In [ ]:
coord = tf.train.Coordinator()

In [ ]:
while not coord.should_stop():
  [...] # do something

In [ ]:
coord.request_stop()

In [ ]:
coord.join(list_of_threads)

In [ ]:
[...] # construction phase
queue_runner = tf.train.QueueRunner(instance_queue, [enqueue_instance] * 5)

with tf.Session() as sess:
  sess.run(enqueue_filename, feed_dict={filename: 'my_test.csv'})
  sess.run(close_filename_queue)
  coord = tf.train.Coordinator()
  enqueue_threads = queue_runner.create_threads(sess, coord=coord, start=True)

In [ ]:
def read_and_push_instance(filename_queue, instance_queue):
  reader = tf.TextLineReader(skip_header_lines=1)
  key, value = reader.read(filename_queue)
  x1, x2, target = tf.decode_csv(value, record_defaults=[[-1.], [-1.], [-1]])
  features = tf.stack([x1, x2])
  enqueue_instance = instance_queue.enqueue([features, target])

In [ ]:
filename_queue = tf.FIFOQueue(capacity=10, dtypes=[tf.string], shapes=[()])
filename = tf.placehodler(tf.string)
enqueue_filename = filename_queue.enqueue([filename])
close_filename_queue = filename_queue.close()
instance_queue = tf.RandomShuffleQueue(capacity=10, min_after_dequeue=2, dtypes=[tf.float32, tf.int32], shapes=[[2], []],
                                       name='instance_q', shared_name='shared_instance_q')

In [ ]:
read_and_enqueue_ops = [
    read_and_push_instance(filename_queue, instance_queue)
    for i in range(5)]

queue_runner = tf.train.QueueRunner(instance_queue, read_and_enqueue_ops)

#Parallelizing

In [ ]:
with tf.Session([...]) as sess:
  [...]
  run_options = tf.RunOptions()
  run_options.timeout_in_ms = 1000
  try:
    pred = sess.run(dequeue_predictions, options=run_options)
  except tf.errors.DeadlineExceededError as ex:
    [...]

In [ ]:
config = tf.ConfigProto()
config.operation_timeout_in_ms = 1000

with tf.Session([...], config=config) as sess:
  [...]
  try:
    pred = sess.run(dequeue_prediction)
  except tf.errors.DeadlineExceededError as ex:
    [...]